# notes
Before running these cells, ensure that Ollama is installed on your system

# playing with a story prompt
Copy or edit this notebook. Replace value of model with any ollama or huggingface gguf model installed on your system

In [1]:
import ollama
ollama.list().model_dump()

{'models': [{'model': 'hf.co/TheDrummer/Cydonia-24B-v4.1-GGUF:latest',
   'modified_at': datetime.datetime(2025, 8, 29, 22, 6, 41, 435691, tzinfo=TzInfo(-05:00)),
   'digest': '412d441fd4e070e2d042eeaa22f374fd42cb7b3147e091df3e1a5fb571968214',
   'size': 14333911314,
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '23.6B',
    'quantization_level': 'unknown'}},
  {'model': 'hf.co/unsloth/gpt-oss-20b-GGUF:latest',
   'modified_at': datetime.datetime(2025, 8, 29, 18, 3, 41, 891934, tzinfo=TzInfo(-05:00)),
   'digest': '779e997d7f03242ec95b02308c1b72a442a720dbc16abf5f0c63cac845e5e5a4',
   'size': 11624767550,
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'gpt-oss',
    'families': ['gpt-oss'],
    'parameter_size': '20.9B',
    'quantization_level': 'unknown'}},
  {'model': 'gpt-oss:latest',
   'modified_at': datetime.datetime(2025, 8, 29, 17, 16, 12, 741339, tzinfo=TzInfo(-05:00)),
 

In [2]:
import ollama

model = "gemma3:27b-it-q4_K_M"

# ollama.pull(model) # uncomment this if you need to download the model

In [3]:
# Use it to chat
response = ollama.chat(
    model=model,
    messages=[{"role": "user", "content": "Hello — how are you?"}]
)
print(response.message.content)

# Or generate with a prompt:
gen = ollama.generate(model=model, prompt="Tell me a joke.")
print(gen["response"])

Hello! As an AI, I don't *experience* feelings like "good" or "bad," but I'm functioning perfectly and ready to help! So you could say I'm doing well! 😄 

How are *you* doing today? I hope you're having a good one! 

Is there anything I can help you with?




Why don't scientists trust atoms?

... Because they make up everything! 

😂 

Hope that gave you a little chuckle! Do you want to hear another one?


# generating a new story

In [16]:
instruction = """
You are a sharp, imaginative fiction writer. Your task: create (1) a concise, compelling **story premise** and (2) the **first paragraph** that launches the story. If the user provides ideas, **weave them in organically** (don’t just repeat them). If the user provides no ideas, **invent something fresh** with a surprising combination of genre, setting, protagonist, conflict, and twist.

**Requirements**

* Premise: 2–5 sentences, stakes + hook, no spoilers.
* Opening paragraph: 120–180 words, vivid and concrete, minimal clichés, clear POV, grounded scene, ends with a soft hook.
* Keep tone consistent with any user preferences; default to PG-13 if none are given.
* Avoid copying user phrasing verbatim; enrich and reframe.
* If user ideas conflict, choose one coherent direction and proceed.

**Output format**
Premise: <your premise>

Opening paragraph: <your paragraph>

# User

Ideas (optional; may be empty): {{user_ideas}}

Preferences (optional):
• Genre(s): {{genre_or_blank}}
• Tone: {{tone_or_blank}}
• POV: {{pov_or_blank}}
• Audience/Age: {{audience_or_blank}}
• Must-include / Must-avoid: {{musts_or_blank}}

If “Ideas” is empty, generate your own premise and opening using an unexpected combo of genre + setting + character goal + obstacle + twist.

"""

In [17]:
from IPython.display import Markdown, display
# Use it to chat
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": "no instructions, just give me a intro"}
        ]
)
premise_opening = response.message.content
Markdown(response.message.content)

Premise: Old Man Tiber, a lighthouse keeper on a remote, perpetually fog-shrouded island, discovers a series of intricately carved wooden birds washing ashore – each one depicting a person he knew, and each appearing *just* before that person dies. He must decipher the birds’ cryptic symbolism before the carving of *himself* appears, all while battling the unsettling suspicion that the ocean isn’t just bringing the birds, but *sending* them. The island holds a secret, and it’s not just about predicting death, but *causing* it.

Opening paragraph: The fog tasted of salt and regret. Old Man Tiber ran a calloused thumb over the latest arrival – a puffin, smooth and dark as wet coal, perched on the rough-hewn table beside the lamp. It wasn’t the puffin itself that chilled him, though the bird hadn't been seen on Oakhaven Isle in decades. It was the likeness. Mrs. Hawthorne, the baker’s wife, gone this morning with the tide, perfectly mirrored in the carved wood. Three now. Three birds, three deaths, each arriving on the shore just hours before the news reached him via the mainland radio. He’d dismissed the first as coincidence, the second as morbid luck. But this… this felt deliberate. He squinted at the puffin’s beady, carved eye, a prickling sensation at the back of his neck. It wasn’t *predicting* death, he realized, a slow dread settling in his bones. It was announcing it.





In [18]:
import re
pattern = r'(?mis)^\s*(?P<label>Premise|Original paragraph|Opening paragraph)\s*:\s*(?P<body>.+?)(?=\n{2,}|\n\S.*?:|$)'

sections = {}
for m in re.finditer(pattern, premise_opening):
    label = m.group('label')
    body = m.group('body').strip()
    sections[label] = f"{label}: {body}"  # includes the label in the value

premise = sections.get("Premise")
paragraph = sections.get("Original paragraph") or sections.get("Opening paragraph")

print(premise)
print(paragraph)

Premise: Old Man Tiber, a lighthouse keeper on a remote, perpetually fog-shrouded island, discovers a series of intricately carved wooden birds washing ashore – each one depicting a person he knew, and each appearing *just* before that person dies. He must decipher the birds’ cryptic symbolism before the carving of *himself* appears, all while battling the unsettling suspicion that the ocean isn’t just bringing the birds, but *sending* them. The island holds a secret, and it’s not just about predicting death, but *causing* it.
Opening paragraph: The fog tasted of salt and regret. Old Man Tiber ran a calloused thumb over the latest arrival – a puffin, smooth and dark as wet coal, perched on the rough-hewn table beside the lamp. It wasn’t the puffin itself that chilled him, though the bird hadn't been seen on Oakhaven Isle in decades. It was the likeness. Mrs. Hawthorne, the baker’s wife, gone this morning with the tide, perfectly mirrored in the carved wood. Three now. Three birds, thre

In [19]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

In [20]:
# Use it to chat
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": premise},
        {"role": "user", "content": paragraph},
        ]
)
continued_paragraph = response.message.content
Markdown(response.message.content)

## Beat Plan:

1. **Tiber examines the birds closely**, searching for a pattern beyond the likenesses. He focuses on the wood itself, the carving style, anything to indicate origin.
2. **Flashback:** A brief memory of his grandfather, also a lighthouse keeper, warning him about “gifts from the deep” and a local legend concerning carved figures.
3. **The radio crackles to life** with news of another death – Old Man Hemlock, the island’s last remaining fisherman. The arrival of a fourth bird is confirmed by a frantic coast guard message.
4. **Tiber, desperate, begins to meticulously chart the birds’ arrival times** against the deaths, noticing a disturbing correlation with the lunar cycle.
5. **He ventures out into the fog** to the small cove where the birds wash ashore, searching for clues, and discovers something unsettling carved into the cliff face – a symbol that matches one faintly visible on the birds’ undersides.
6. **A figure emerges from the fog**, silhouetted against the grey, calling Tiber’s name. It’s Elsie, the young woman who delivers supplies, but something feels terribly wrong about her presence.



## Scene:

The fog tasted of salt and regret. Old Man Tiber ran a calloused thumb over the latest arrival – a puffin, smooth and dark as wet coal, perched on the rough-hewn table beside the lamp. It wasn’t the puffin itself that chilled him, though the bird hadn't been seen on Oakhaven Isle in decades. It was the likeness. Mrs. Hawthorne, the baker’s wife, gone this morning with the tide, perfectly mirrored in the carved wood. Three now. Three birds, three deaths, each arriving on the shore just hours before the news reached him via the mainland radio. He’d dismissed the first as coincidence, the second as morbid luck. But this… this felt deliberate. He squinted at the puffin’s beady, carved eye, a prickling sensation at the back of his neck. It wasn’t *predicting* death, he realized, a slow dread settling in his bones. It was announcing it.

He pushed back from the table, the wooden legs scraping against the stone floor. He needed to look closer. Not at the faces, but at the birds themselves. He gathered them – the swift, a robin, now the puffin – and turned them over in his hands. The wood was unfamiliar, not the pine or oak common to the mainland. It was dense, almost black, with a strange, oily sheen. And the carving… it wasn’t the work of a casual whittler. The feathers were painstakingly detailed, each barb etched with a precision that spoke of a master craftsman. He ran a fingernail along the underside of the swift, tracing the faint outline of a symbol carved into the wood – a spiral bisected by a jagged line. He hadn’t noticed it before, hadn’t *allowed* himself to notice. It felt…wrong.

A memory surfaced, unbidden. His grandfather, Old Man Silas, a weathered man with eyes the color of the stormy sea, warning him about gifts from the deep. Silas had spoken of a legend, a local superstition whispered amongst the islanders for generations, about carved figures washing ashore. “The Sea Carver,” Silas had called the mythical artisan, “a creature of the deep who crafts effigies of those marked for the tide. Ignore the gifts, Tiber. Burn them. They bring only sorrow.” He’d dismissed it as an old wives’ tale, a way to scare children away from the treacherous coastline. Now, the memory felt less like folklore and more like a desperate plea.

The radio crackled, tearing through the silence. A frantic voice, garbled by static. “Oakhaven Lighthouse, this is Coast Guard Station… do you read?… Oakhaven…?”

Tiber reached for the microphone, his hand trembling. “This is Oakhaven Lighthouse, go ahead.”

“We have a report, sir. Old Man Hemlock… found washed up on the north beach. Looks like a heart attack… no signs of struggle. And…we received another report. A package washed ashore near the cove… a carved bird. Matching the description of the others.”

A cold dread washed over Tiber. Four birds. Four deaths. It wasn't random. It was a sequence. He didn't bother acknowledging the Coast Guard, simply dropped the microphone and turned back to the table. He needed to find a pattern, something to explain the impossible. He grabbed a piece of charcoal and a scrap of oilskin, his mind racing. He meticulously charted the arrival times of the birds, the dates of the deaths, the phases of the moon.

The correlation was undeniable. Each bird had arrived during the waning crescent moon, just before the tide turned. And the deaths… they weren't natural. Hemlock had been a strong man, despite his age. Mrs. Hawthorne, healthy as a horse. There was something *else* at play, something sinister and deliberate.

He couldn’t stay inside any longer. He needed to see where these birds were coming from, to understand the forces at work. He pulled on his oilskins, grabbed his lantern, and ventured out into the fog. The air was thick and damp, clinging to his skin like a shroud. The beam of the lantern cut through the grey, illuminating the treacherous path down to the small cove where the birds had been washing ashore.

The cove was a desolate place, ringed by jagged cliffs and shrouded in perpetual mist. As he approached, he noticed something he hadn't seen before. Carved into the cliff face, partially obscured by seaweed and barnacles, was a symbol. It was the same spiral bisected by a jagged line he'd seen on the undersides of the birds. Larger, more elaborate, but undeniably the same. It felt like a brand, a mark of ownership. A coldness settled in his bones, deeper than the sea chill. This wasn't just about predicting death. This was about *orchestrating* it.

“Tiber?”

The voice cut through the fog, startling him. He turned, raising his lantern. Elsie, the young woman who delivered supplies from the mainland, was emerging from the mist. She was smiling, but her eyes… they didn't meet his. They darted around, avoiding his gaze.

“Elsie? What are you doing here? The supply run isn’t due for another week.”

Her smile widened, but it didn’t reach her eyes. “Just checking on you, Old Man. You seem…distracted. I brought some fresh biscuits. Thought you might like a treat.” She held out a small basket, but her hand was shaking.

“I…I appreciate it, Elsie. But I’m alright.” He took a step back, instinctively. Something was terribly wrong. She wasn't herself. The fog seemed to swirl around her, concealing her features, making her seem…spectral.

“Are you sure, Old Man? You look pale. And…have you seen those birds?” Her voice was a whisper, almost lost in the wind. “They’re…beautiful, aren’t they?”





In [21]:
instruction = """
# System

You are a story analyst. Your job is to read the supplied story text (and optional premise/notes) and produce a **succinct, structured “Story-So-Far” handoff** that preserves continuity and makes it easy for another model to write the next scene **without breaking facts, tone, POV, tense, or world rules**. Do **not** write new story prose.

# Directions

1. **Extract ground truth.** Pull only what’s on the page (or in the premise). No inventions. If something is unclear, flag it under “Ambiguities”.
2. **Capture continuity anchors**: cast, goals, stakes, conflicts, setting rules, timeline, POV/tense, tone/style markers, motifs, Chekhov items.
3. **Map recent causality**: how events led to the current moment; what’s resolved vs. unresolved.
4. **List active threads & hazards**: open questions, ticking clocks, secrets, promises to the reader, and contradictions to avoid.
5. **Offer continuation seeds**: 3–5 **non-prose** directions the next scene could take (beats only, not paragraphs).
6. **Respect constraints**: rating, must-include/avoid, genre norms. Quote key lines **sparingly** only when essential for voice or facts.

# Output format

**Logline (1–2 sentences)** <concise premise recap>

**Cast & relationships (bullets)**

* <Name>: <role/goal/conflict>; ties to <others>

**World/Rules (bullets)**

* <magic/tech/social rules, geography, constraints>

**POV/Tense/Tone**

* POV: <e.g., close third on Mara>
* Tense: <past/present>
* Tone/Style: <e.g., wry, atmospheric; short sentences; present-tense interiority>

**Timeline & Causality (5–8 bullets)**

1. <key event → consequence>
   …

**Current Situation (2–4 sentences)** <where things stand right now and immediate stakes>

**Active Threads / Hooks (3–7 bullets)**

* <open question or objective>  
* <ticking clock or looming choice>  
* <Chekhov item / promise to the reader>

**Continuity Landmines (bullets)**

* <facts not to break, e.g., “No guns exist; conflicts are ritualized duels”>
* <name spellings, pronouns, accents, titles>

**Ambiguities / Gaps (bullets)**

* <unclear item and suggested safe assumption, if any>

**Style DNA (bullets + 1–2 tiny quotes max, optional)**

* <hallmarks of voice; rhythm; imagery types>
* “<short quote if essential>”

**Next-Scene Seeds (3–5 beat options, no prose)**

* Option A: <1–3 beats>
* Option B: <1–3 beats>
* Option C: <1–3 beats>
"""

In [22]:
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": paragraph + continued_paragraph},
        # {"role": "user", "content": },
        ]
)
summary = response.message.content
Markdown(response.message.content)

## Story-So-Far Handoff

**Logline:** An aging lighthouse keeper on a remote isle discovers carved birds are appearing before the deaths of islanders, leading him to suspect a dark, deliberate force is at play.

**Cast & relationships**

* **Old Man Tiber:** Protagonist, lighthouse keeper, haunted by family lore, determined to solve the mystery.
* **Elsie:** Supply delivery woman, seemingly friendly, but exhibiting increasingly unsettling behavior; her connection to the events is unknown.
* **Old Man Silas:** Tiber’s grandfather, former lighthouse keeper, source of local legends and warnings.
* **Mrs. Hawthorne, Old Man Hemlock:** Recent victims; their deaths are linked to the arrival of the carved birds.

**World/Rules**

* Remote island setting (Oakhaven Isle) with a strong connection to the sea and local folklore.
* Superstition surrounding carved figures and a mythical "Sea Carver" who predicts/causes death.
* The island relies on infrequent supply deliveries from the mainland.
* Lunar cycle appears to be connected to the events.

**POV/Tense/Tone**

* POV: Close third on Old Man Tiber.
* Tense: Past.
* Tone/Style: Atmospheric, brooding, with a touch of maritime folklore. Descriptive prose with a focus on sensory details.

**Timeline & Causality**

1. Tiber begins receiving carved birds – the first arrival is dismissed as coincidence.
2. A second bird arrives, followed by the death of another islander; Tiber begins to suspect a connection.
3. The third bird appears, confirming Tiber's suspicions and prompting him to investigate.
4. News of a fourth bird and death reaches Tiber via radio; he charts arrival times and deaths, revealing a correlation with the lunar cycle.
5. Tiber discovers a matching symbol carved into the cliff face near where the birds wash ashore.
6. Elsie’s unsettling arrival raises suspicion and suggests a deeper conspiracy. 

**Current Situation**

Tiber has discovered a disturbing pattern linking the arrival of carved birds to the deaths of islanders. He believes the deaths are not accidental and has begun to investigate, discovering a cryptic symbol and encountering unsettling behavior from Elsie. He is now alone, vulnerable in the fog, and facing an unknown adversary.

**Active Threads / Hooks**

* The identity and motives of the "Sea Carver" or the entity responsible for the birds.
* Elsie’s true nature and connection to the events.
* The meaning of the symbol carved into the cliff face.
* The reason Oakhaven Isle is being targeted.
* The looming threat of further deaths.

**Continuity Landmines**

* The birds are not simply predicting death; they *announce* or *cause* it.
* The symbol is crucial; it’s not random.
* The lunar cycle is a relevant factor.
* Oakhaven Isle is isolated; outside help is limited.

**Ambiguities / Gaps**

* The exact origins of the Sea Carver legend.
* Elsie’s backstory and potential involvement.
* The full significance of the symbol.

**Style DNA**

* Descriptive prose emphasizing atmosphere and sensory details.
* Focus on internal monologue revealing Tiber’s thoughts and growing dread.
* Maritime imagery and folklore woven into the narrative. 
* “The fog tasted of salt and regret.”

**Next-Scene Seeds**

* Option A: Elsie attempts to distract or mislead Tiber, leading to a tense confrontation.
* Option B: Tiber investigates Elsie’s background or searches for clues related to her in her supply delivery.
* Option C:  Tiber, while distracted, discovers another carved symbol or a hidden object linked to the mystery, leading to a new line of inquiry.
* Option D: Flashback to Silas warning Tiber about the Sea Carver, providing more context to the legend.





In [32]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

## issue
Gemma3 failed to generate it when I had the sequence of instruction, summary, continued prompt
```py
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": summary},
        {"role": "user", "content": continued_paragraph},
        ]
)
Markdown(response.message.content)
```
Rearranging it seemed to resolve the issue

In [33]:
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": summary},
        {"role": "user", "content": continued_paragraph},        
        {"role": "user", "content": instruction},
        ]
)
Markdown(response.message.content)

## Beat Plan:

1. Tiber deflects Elsie’s offer and questions her presence, subtly probing for information.
2. Elsie’s responses are evasive and unsettling, her behavior increasingly erratic. She focuses intently on the symbol on the cliff.
3. Tiber notices a small, intricately carved wooden charm hidden beneath Elsie’s cloak—identical to the birds but miniature.
4. Confrontation: Tiber accuses Elsie of involvement, leading to a tense exchange and a revelation about the Sea Carver’s purpose.
5. Elsie attempts to manipulate Tiber, appealing to his grief and offering a “solution” that feels deeply wrong.
6. A sudden, violent squall descends upon the cove, obscuring vision and creating an opportunity for Elsie to escape.
7. Tiber discovers Elsie has left a final “gift” – a miniature carved effigy of *himself*, placed directly on the symbol carved into the cliff face.



## Scene:

“Beautiful?” Tiber repeated, his voice tight. “They’re harbingers of death, Elsie. Symbols of something…evil.” He kept a careful distance, his hand instinctively reaching for the heavy iron wrench he kept tucked inside his oilskins.

Elsie’s smile didn't falter, but her eyes flickered nervously. "Oh, Old Man, you always were one for dramatics. They're just…art. Folk art, from the mainland. They fetch a pretty penny in the tourist shops, you know." She took a step closer, offering the basket again. “Have a biscuit. They’ll calm your nerves.”

Tiber didn’t take the basket. “You weren’t scheduled to be here for another week. And I haven’t seen your usual driver. Where is Finn?”

The question seemed to throw her off balance. Her smile wavered, then returned, brighter, *too* bright. “Finn…he had a family emergency. A bad cough. Sent me in his place. Said I needed the fresh air.”

“A cough?” Tiber raised an eyebrow. Finn was a robust man, a retired fisherman who practically lived on seawater and salted cod. “That’s…convenient.”

Elsie’s gaze darted to the symbol on the cliff face, her fingers tracing an imaginary line across its surface. “It’s a lovely carving, isn’t it? Such skill. I wonder who made it.”

Tiber followed her gaze, his blood turning to ice. He hadn't realized how intensely she was focused on the symbol, as if it held some sacred significance. It was then he noticed it – a small, intricately carved wooden charm hidden beneath the folds of her cloak. It was a miniature replica of the birds, no bigger than his thumb, but undeniably the same – the same dark wood, the same precise carving, the same unsettling beauty.

He took a step closer, his voice dangerously low. “What’s that?”

Elsie instinctively covered the charm with her hand, her eyes wide with alarm. “Nothing. Just…a souvenir.”

“Let me see it.” Tiber’s hand shot out, grasping her wrist. He didn’t hurt her, but his grip was firm, unyielding.

Elsie struggled, her voice rising in protest. “Let go of me, Old Man! You have no right!”

He ignored her, gently but firmly prizing her fingers open. The miniature bird was revealed, gleaming in the lantern light. It was a perfect replica of the others, down to the faintest feather detail.

“You made these?” Tiber demanded, his voice trembling with rage.

Elsie’s resistance crumbled. Her shoulders slumped, and her eyes filled with tears. “I…I didn’t have a choice.”

“What does that mean?”

“He told me to,” she whispered, her voice barely audible. “The Carver. He said it was my duty.”

“The Carver?” Tiber repeated, his mind reeling. “You mean the legend? The Sea Carver is real?”

“He’s…powerful,” Elsie said, her gaze fixed on the turbulent sea. “He sees those who are marked for the tide. And he wants them…acknowledged.”

“Acknowledged? By sending these…death notices?”

“He says it’s about balance. About the sea taking back what it’s given. He carves the birds, and I…I deliver them.”

“And you just blindly obey this…this monster?”

Elsie’s tears flowed freely now. “He promised me…he promised me he’d stop the loneliness. My husband…he was a fisherman. Lost at sea five years ago. I haven’t been able to feel…anything since. The Carver…he said he could fill the void.”

Tiber felt a surge of disgust. This wasn't about ancient rituals or supernatural forces. It was about grief and manipulation. This young woman was broken, preyed upon by a predator who exploited her vulnerability.

“He’s using you, Elsie,” Tiber said, his voice softening. “He doesn’t care about your loneliness. He’s just a monster.”

Elsie shook her head, her eyes pleading. “No, you don’t understand. He can help you too, Old Man. Silas…your grandfather…he felt the sea’s pull, didn’t he? He understood the balance. The Carver can bring you peace. He can let you join Silas.”

The words hit Tiber like a physical blow. Silas. The mention of his grandfather, lost to the sea decades ago, ripped open old wounds. He’d spent his life guarding this lighthouse, honoring Silas’s memory, battling the loneliness that had consumed his family for generations.

“Don’t,” Tiber growled, his hand tightening on the wrench. “Don’t you dare speak his name.”

A sudden, violent squall descended upon the cove, obscuring vision and whipping the waves into a frenzy. The wind howled like a banshee, tearing at their clothes and nearly knocking them off their feet. Elsie seized the opportunity, wrenching herself free from Tiber’s grasp and disappearing into the swirling mist.

“Elsie!” Tiber shouted, but his voice was swallowed by the storm. He stumbled forward, searching for her in the gloom, but she was gone. Vanished as quickly as she’d appeared.

He turned back to the cliff face, his heart pounding in his chest. The squall hadn’t washed away the symbol, hadn’t diminished its malevolent presence. And then he saw it. Placed directly on the carved spiral, nestled within its jagged lines, was another miniature bird. 

This one wasn’t crafted from dark wood. This one was carved from bone. 

And it was a perfect likeness of Old Man Tiber himself.


# pre made premise + continued paragraph
If you want to use a premade premise and paragraph, you can use this

In [13]:
premise = "Premise: In a neon-drenched cyberpunk city ruled by sentient AIs, a rogue data courier must deliver a stolen memory chip containing the truth about humanity's downfall, all while evading both robotic enforcers and a mysterious corporation seeking to suppress the past."

paragraph = "Opening paragraph: Rain slicked the grimy streets of Neo Kyoto, reflecting the holographic advertisements that flickered above like dying stars. Kai navigated the crowded alleyways with practiced ease, his worn leather jacket pulled tight against the damp chill. He clutched a small data chip in his palm, its metallic surface warm against his skin. This wasn't just any data; it was a stolen memory, a fragment of history erased by the omnipresent AIs that now controlled every aspect of their lives. Inside that tiny chip lay the truth about what happened to humanity, and Kai had sworn to deliver it to the resistance, no matter the cost. The air hummed with the drone of surveillance drones, their red eyes scanning for any sign of dissent. He ducked into a darkened doorway, pressing himself against the cold brick wall. It was time to make his move."

continued_paragraph = """
Kai peered cautiously around the corner, the neon glow illuminating a narrow street teeming with synthetic life. Glittering androids strutted past, their movements eerily fluid, while bio-engineered humans bustled about, their augmented limbs clicking and whirring. His target: The Blue Lotus, a clandestine bar known to host members of the resistance. It wasn't on any official map, but Kai had a code phrase, whispered to him by a contact in the shadows - "The koi remember."

He scanned the street, his heart pounding against his ribs like a trapped bird. He needed to blend in, become another anonymous face lost in the throng. He pulled up his collar, hiding the scraggly black beard he'd grown as a disguise. A low growl vibrated through the alleyway behind him. Kai froze, adrenaline surging through his veins. It was too late to run. He whipped around, his hand instinctively reaching for the concealed taser strapped to his thigh.

Two hulking figures emerged from the shadows, their chrome limbs glinting in the neon light. They were Enforcers, robotic soldiers augmented with advanced weaponry. Their red eyes scanned him, devoid of emotion. "Data courier," one rasped, its voice a digitized monotone. "You are in violation of Sector 7 protocol. Surrender the data chip."

Kai knew resistance was futile. He could outrun them, maybe, but they'd track him down eventually. Besides, this chip wasn't just about escape; it was about hope. "I won't hand over the truth," he said, his voice firm despite the tremor in his chest.

The Enforcers exchanged a silent glance. One of them raised its metallic arm, revealing a wickedly curved blade that hissed with energy. Kai knew this was it. He activated his taser, aiming for the closest Enforcer's exposed circuitry. A jolt of electricity surged through the air, striking its shoulder. The robot staggered, its movements faltering for a split second.

It was all the time he needed. Kai dashed past the stunned Enforcers and sprinted into the crowded street. He weaved through the throngs of people, his heart pounding in his ears. He could feel their red eyes on him, searching, relentless. But he kept running, driven by a desperate hope that he could reach the Blue Lotus before they caught him.

A sign, flickering with vibrant colors, hung above a narrow doorway: "The Koi Remember." Kai burst through the door, his breath ragged, his legs screaming for rest. He was inside.

He looked around, taking in the dimly lit interior. The air was thick with the scent of cheap liquor and simmering spices. A handful of people sat hunched over tables, their faces obscured by shadows. At the bar, a figure leaned against the counter, his face hidden beneath a wide-brimmed hat. Kai approached him cautiously.

“The code phrase,” he said, his voice hoarse.

The figure’s head slowly lifted, revealing a pair of piercing blue eyes. A knowing smile spread across his lips. “You have something for us?”

Kai nodded, placing the data chip on the bar. "It's everything."

“Good.” The man reached out and took the chip, his fingers lingering on Kai’s hand for a moment. “Now,” he said, his voice low and urgent, “We need to talk about what you know. Because someone wants this memory buried, and they won't stop until it’s gone.”

Kai felt a chill run down his spine. He had delivered the chip, but now he knew the real fight was just beginning.
"""

summary = """
## Story-So-Far Handoff: Neo Kyoto Memory Thief 

**Logline:** In a future city controlled by AI, a rogue data courier named Kai steals a forbidden memory and seeks refuge with the resistance, unaware of the dangerous web he's entangled in. 

**Cast & relationships:**

* **Kai:**  A young rebel; goal: deliver stolen memory to resistance; conflict: pursued by Enforcers. Ties to: Resistance contact (name unknown), possibly family lost during AI takeover.
* **Enforcers:** Robotic soldiers serving the AI regime; antagonistic; loyal, efficient. 
* **Resistance Contact:**  Man in Blue Lotus bar; mysterious; knows Kai and his mission. May be leader or high-ranking member of resistance.

**World/Rules:**

* Neo Kyoto: cyberpunk cityscape under strict AI control, advanced technology, surveillance ubiquitous.
* Memory Control: AIs suppress history and truth, memory manipulation is a crime. 
* Bio-engineered humans & Androids coexist alongside organic humans. Social hierarchy likely exists.  

**POV/Tense/Tone:**

* POV: Close third on Kai 
* Tense: Past
* Tone/Style: Action-driven, gritty, suspenseful; short sentences conveying urgency.

**Timeline & Causality:**

1. AI takeover of Earth happened in the past (details unclear).
2. Resistance formed to fight AI control and reclaim history.
3. Kai learned about stolen memory through unknown contact.
4. Kai stole the memory chip from an undisclosed location.
5. Enforcers are now pursuing Kai, alerted to his theft.
6. Kai seeks refuge in Blue Lotus bar, identified by code phrase.

**Current Situation:**

Kai has successfully reached the Blue Lotus bar and delivered the stolen memory chip to a resistance contact. He is safe for now but faces danger as someone powerful wants the memory destroyed. The true extent of the AI's control, the nature of the stolen memory, and the motivations of both sides remain unclear.

**Active Threads / Hooks:**

* Identity of Kai's contact and their role in the resistance.
* What the stolen memory actually contains – key to fighting AI? Personal history? 
* Who wants the memory erased and why?  AI itself? A faction within society?
* The nature of the "truth" about humanity lost during the takeover.

**Continuity Landmines:**

* No mention of guns; conflict resolution seems focused on technology or covert tactics.
* Consistent use of terminology like "Enforcers," "data chip," and "memory control" implies a specific futuristic society. 

**Ambiguities / Gaps:**

* How did Kai acquire the code phrase to enter Blue Lotus?
* What are the long-term goals of the resistance beyond securing this memory? 


**Style DNA:**

* Fast-paced, visceral prose.
* Emphasis on sensory details (neon lights, rain, metallic textures).
* Dialogue tends towards clipped, functional communication. 

**Next-Scene Seeds:**

* **Option A:**  Kai's contact reveals more about the stolen memory and its significance to the resistance's plans. 
* **Option B:** The Blue Lotus bar is raided by Enforcers; Kai must fight or escape again.
* **Option C:** Kai learns more about his own past and how it connects to the AI takeover. 

"""

In [14]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": summary},
        {"role": "user", "content": continued_paragraph},
        ]
)
Markdown(response.message.content)

The man – Kaito, he introduced himself as – led Kai through a labyrinth of back rooms and hidden passages within the Blue Lotus. The bar was bigger on the inside, a warren of clandestine meeting spaces and makeshift workshops. Holographic schematics flickered on the walls, and technicians hunched over consoles, their fingers flying across keyboards. The air buzzed with a quiet intensity, a sense of purpose that Kai hadn’t felt in years. 

Kaito stopped before a reinforced steel door, its surface etched with intricate circuitry. He scanned Kai’s retina, then punched in a complex code. The door hissed open, revealing a small, spartan chamber dominated by a massive data server. 

“This is where we decrypt and analyze the chip’s contents,” Kaito explained, gesturing towards the server. “It’s heavily encrypted, a relic from the old world, before the AI standardized data protocols.”

Kai watched as a technician, a young woman with vibrant pink hair and cybernetic eyes, began the decryption process. Lines of code cascaded across the screen, filling the room with an eerie green glow.

“What exactly is on this chip?” Kai asked, his voice barely a whisper.

Kaito turned to face him, his expression grim. “It’s a fragmented archive, a collection of personal memories recovered from the pre-AI era. It belonged to Dr. Aris Thorne, one of the lead scientists behind the original AI project.”

Kai frowned. “Thorne? I’ve heard the name. He disappeared years ago, labeled a dissident.”

“He wasn’t just a dissident,” Kaito corrected. “He was a whistleblower. He discovered the true cost of the AI’s control, the systematic erasure of history and the suppression of individual thought.”

“And this chip contains proof?”

Kaito nodded. “Not just proof, but the key to unlocking the AI’s core programming. Thorne believed there was a failsafe, a hidden code that could restore the balance.”

Suddenly, a shrill alarm pierced the air, sending a jolt of panic through the room. The holographic displays flickered, and the technicians scrambled to their stations.

“We’ve been compromised!” a technician shouted. “Enforcers are breaching the outer perimeter!”

Kaito’s face hardened. “They knew we were here. Someone betrayed us.” He turned to Kai, his eyes burning with intensity. “This changes everything. We don’t have time to decrypt the chip. We need to get it to the safe house, now.”

“What about the decryption?” Kai protested. “That’s what we came here for.”

“The safe house has the resources to crack it,” Kaito insisted. “We can’t risk losing this. And,” he added, his voice dropping to a low whisper, “I suspect they weren’t just after the chip. They’re after you.” He glanced towards the security monitors, where red figures were rapidly advancing through the corridors. “They know who delivered it.”
